In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReviewAnalysis").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://tflynn615-vine.s3.us-east-2.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True)

df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11X0I2GAJ7ZHO|          1|            0|          0|   N|                Y|
|R1QCGNX8V8U7H5|          5|            0|          0|   N|                Y|
|R1XTKENV68TZZF|          5|            0|          0|   N|                Y|
|R1MWFEB4B4DOPF|          5|            1|          1|   N|                Y|
|R122JEYIN5238V|          1|            2|          2|   N|                Y|
|R3S0UVTM8WOEAJ|          5|            0|          0|   N|                Y|
| R8BUSTIR4G4IU|          5|            0|          0|   N|                N|
| RWG9TO53TW6UT|          3|            1|          1|   N|                Y|
|R2M8RW0HAMPBJD|          3|            1|          1|   N|                Y|
|R34X8V8SGSZX2S|          4|            0|          0|   N|     

In [4]:
total_df = df.filter("total_votes>20")
total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2O2MVQWXRBEAF|          2|           19|         22|   N|                Y|
|R2D1I6BW7RLRJR|          3|           38|         43|   N|                Y|
|R29EPD5BP4MOIC|          1|           17|         30|   N|                Y|
|R13H6BIQLHBG9J|          3|           22|         22|   N|                Y|
| RIXWNTR8D8AA9|          5|           60|         61|   N|                Y|
|R15P6YTD54R2WL|          5|           21|         24|   N|                Y|
|R2PR3OB794LC9L|          3|           35|         43|   N|                Y|
|R310XMILWS4RPO|          4|           41|         42|   N|                Y|
|R3VU0RR422AJDO|          5|           70|         78|   N|                Y|
| REO3UTDOPYIH3|          4|           21|         23|   N|     

In [5]:
helpful_df = total_df.filter("helpful_votes/total_votes>=0.5")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2O2MVQWXRBEAF|          2|           19|         22|   N|                Y|
|R2D1I6BW7RLRJR|          3|           38|         43|   N|                Y|
|R29EPD5BP4MOIC|          1|           17|         30|   N|                Y|
|R13H6BIQLHBG9J|          3|           22|         22|   N|                Y|
| RIXWNTR8D8AA9|          5|           60|         61|   N|                Y|
|R15P6YTD54R2WL|          5|           21|         24|   N|                Y|
|R2PR3OB794LC9L|          3|           35|         43|   N|                Y|
|R310XMILWS4RPO|          4|           41|         42|   N|                Y|
|R3VU0RR422AJDO|          5|           70|         78|   N|                Y|
| REO3UTDOPYIH3|          4|           21|         23|   N|     

In [7]:
vine_df = helpful_df.filter(helpful_df["vine"]=='Y')
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2012Y5S3OSZIC|          5|           89|         92|   Y|                N|
|R3OFJBZGLZIUZN|          5|           23|         25|   Y|                N|
|R19IMEOXPFYTYF|          4|           58|         64|   Y|                N|
|R14WO7F0WU7UUH|          5|           43|         52|   Y|                N|
| R9SQ52SZTDU9F|          5|          115|        133|   Y|                N|
|R24DWRW5KISRB6|          5|           30|         33|   Y|                N|
| RR4BE0JTFZPFN|          3|           50|         52|   Y|                N|
|R3I71LOJ4DKK6M|          5|           30|         34|   Y|                N|
|R2CHC193Y2IQRU|          5|           52|         57|   Y|                N|
|R2EYKK32WQLLFM|          4|           22|         24|   Y|     

In [8]:
not_vine_df = helpful_df.filter(helpful_df["vine"]=='N')
not_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2O2MVQWXRBEAF|          2|           19|         22|   N|                Y|
|R2D1I6BW7RLRJR|          3|           38|         43|   N|                Y|
|R29EPD5BP4MOIC|          1|           17|         30|   N|                Y|
|R13H6BIQLHBG9J|          3|           22|         22|   N|                Y|
| RIXWNTR8D8AA9|          5|           60|         61|   N|                Y|
|R15P6YTD54R2WL|          5|           21|         24|   N|                Y|
|R2PR3OB794LC9L|          3|           35|         43|   N|                Y|
|R310XMILWS4RPO|          4|           41|         42|   N|                Y|
|R3VU0RR422AJDO|          5|           70|         78|   N|                Y|
| REO3UTDOPYIH3|          4|           21|         23|   N|     

In [18]:
# Total Paid 

total_paid = vine_df.count()

print(total_paid)

926


In [19]:
# Total Unpaid 

total_unpaid = not_vine_df.count()

print(total_unpaid)

41096


In [20]:
# Number of 5-Star Reviews Paid 

five_paid = vine_df.filter("star_rating=5").count()
print(five_paid)

408


In [21]:
# Number of 5-Star Reviews Paid 

five_unpaid = not_vine_df.filter("star_rating=5").count()
print(five_unpaid)

18104


In [22]:
# Percentage of Reviews 5 stars 

paid_percent = (five_paid/total_paid)*100 
print(paid_percent) 
unpaid_percent = (five_unpaid/total_unpaid)*100 
print(unpaid_percent)

44.06047516198704
44.052949192135486
